# Welcome to the Live Demo of Franken 

## Step 1 - Training our franken MLIP.  

### Import and installation

We start by installing on this (if on google collab) franken with the [mace] option for using MACE-based models. 
This can be done by using simply pip:

In [1]:
!pip install franken[mace]

After installing, we will import all necessary modules for running our example (everythign should be installed along with franken)

In [2]:
from pathlib import Path
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import ase
import ase.md
import ase.md.velocitydistribution
from ase.geometry.rdf import get_recommended_r_max, get_rdf

from franken.calculators.ase_calc import FrankenCalculator
from franken.datasets.registry import DATASET_REGISTRY
from franken.backbones.utils import CacheDir
from franken.autotune import autotune
from franken.config import (
    MaceBackboneConfig,
    MultiscaleGaussianRFConfig,
    DatasetConfig,
    SolverConfig,
    HPSearchConfig,
    AutotuneConfig,
)

c:\Users\pedro\Desktop\Projects\franken-demo\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Autotune - Franken's easy optimization

Franken is equipped with it's own easy and parallel hyperparameter optimizer called Autotune. This module runs the search over the specified hyperparams and for the given number of trials. 

#### Setup configurations (GNN, HEAD and DATA)

From the API we imported the necessary modules for running franken. (remember the whole training can also be done via CLI)
To prepare the training, we need to specify our configurations.
This step prepares the configurations for the custom Backbone (MACE) class, dataset class and the multiscale gaussian random features head.

In [3]:
#Setup Configurations for the MACE backbone, dataset, and multiscale Gaussian RF head

#GNN configuration for MACE backbone
gnn_config = MaceBackboneConfig(
    path_or_id="MACE-L0", #Here we use the model ID in our registry, but you can also use a path to a local model
    interaction_block=2, # Number of interaction blocks in this case L1 on the MACE model
    )

#Dataset configuration for our registered water dataset 
dataset_cfg = DatasetConfig(
    name="water", # Name of the dataset to use, registered in DATASET_REGISTRY or a custom dataset path
    max_train_samples=8, # Maximum number of training samples to use, we will use only 8 samples for this demo
    )

#Random features head configuration for the multiscale Gaussian RF
rf_config = MultiscaleGaussianRFConfig(
    num_random_features=512,    # Number of random features to use 
    length_scale_low=10, # Minimum length scale value
    length_scale_high=30, # Maximum length scale value
    length_scale_num=2, # Number of length scales to try (trials within low and high range)
    rng_seed=42,  # for reproducibility
)


This is convenient because thanks to our registry, setting the dataset configuration to "water" will make it download the water dataset automatically and spceifying the "MACE-L0" will make it download the checkpoint as well. 

#### Set up Autotune's configuration

In [4]:
#Configuration for the Autotune's solver to use 
solver_cfg = SolverConfig(
    l2_penalty=HPSearchConfig(start=-10, stop=-5, num=3, scale='log'),  # L2 Penalty for generalization (equivalent of numpy.logspace)
    force_weight=HPSearchConfig(start=0.01, stop=0.99, num=3, scale='linear'),  # (alpha) weight coefficient on the Loss (equivalent of numpy.linspace)
)

#Setting up Autotune by passing all needed configs
autotune_cfg = AutotuneConfig(
    dataset=dataset_cfg, #Dataset config
    solver=solver_cfg, #Solver config
    backbone=gnn_config, #Backbone GNN config
    rfs=rf_config, #RF Head config
    seed=42, #Seed for reproducibility
    jac_chunk_size=4, #Depends on your GPU'scapabilities 
    run_dir="./results", #Output directory 
)

#### Running the training task

In [10]:
#Returns the output path
run_path = autotune(autotune_cfg)

console_logging_level: INFO
dtype: float64
jac_chunk_size: 4
rf_normalization: leading_eig
run_dir: ./results
save_every_model: False
save_fmaps: False
scale_by_species: True
seed: 42
backbone:
    family: mace
    interaction_block: 2
    path_or_id: MACE-L0
dataset:
    max_train_samples: 8
    name: water
    test_path: null
    train_path: null
    val_path: null
rfs:
    length_scale_high: 30
    length_scale_low: 10
    length_scale_num: 2
    num_random_features: 512
    rf_type: multiscale-gaussian
    rng_seed: 42
    use_offset: true
solver:
    force_weight:
      num: 3
      scale: linear
      start: 0.01
      stop: 0.99
      value: null
      values: null
    l2_penalty:
      num: 3
      scale: log
      start: -10
      stop: -5
      value: null
      values: null

2025-07-14 10:32:04.116 WARNING (rank 0): Cache directory already initialized at C:\Users\pedro\.franken. Reinitializing.
2025-07-14 10:32:04.116 WARNING (rank 0): Cache directory already initialized at 

ConnectTimeout: HTTPSConnectionPool(host='zenodo.org', port=443): Max retries exceeded with url: /api/records/10723405/files-archive (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002874F8721A0>, 'Connection to zenodo.org timed out. (connect timeout=None)'))

## Step 2 - Loading Franken as an ASE calculator. 

Franken comes with an implementation on ASE (FrankenCalculator) and a compiling script for running MD on LAMMPS (franken.create_lammps).
Since we want to stick to Python, we will use ASE to run some MD. 

#### Load with Franken calculator

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" #Use cuda for fastest inference 
model_path = run_path / "best_ckpt.pt"
calc = FrankenCalculator(model_path, device=device)


#### Validate E/F predictions

## Step 3 - Running MD with Franken

Running MD with franken is as easy as loading the ASE calculator and setting up an MD scheme. 

#### Setup MD simulation and parameters

In [ ]:
md_length_ns = 0.01 #Small simulation
timestep_fs = 0.5 #Simulation's timestep, for this light model we will set it small
temperature_K = 300 # T in Kelvin
friction = 0.01 #Friction of the Langevin integrator


In [ ]:
#Fetching starting configuration from the downloaded data
data_path = DATASET_REGISTRY.get_path("water", "train", CacheDir.get())
initial_configuration = ase.io.read(data_path, index=567)

#Setting up MD's path
md_path = run_path / "md"
md_path.mkdir(exist_ok=True)

# Trajectory will contain the output data
trajectory_path = md_path / f"md_output.traj"
trajectory = ase.io.Trajectory(trajectory_path, "w", initial_configuration)

#Assigning calculator
initial_configuration.calc = calc

#### Running the actual MD 

In [ ]:
#Initialize velocities
ase.md.velocitydistribution.MaxwellBoltzmannDistribution(
    atoms=initial_configuration,
    temperature_K=temperature_K
)

#Initialize integrator
integrator = ase.md.Langevin(
    atoms=initial_configuration,
    temperature_K=temperature_K,
    friction=friction / ase.units.fs,
    timestep=timestep_fs * ase.units.fs
)

#Setting up reporter and attaching it 
md_logger = ase.md.MDLogger(
    integrator,
    initial_configuration,
    sys.stdout,  # log directly to console
    header=True,
    stress=False,
    peratom=False,
)
integrator.attach(md_logger, interval=100)
integrator.attach(trajectory.write, interval=100)

In [ ]:
#Run for the number of steps
integrator.run(md_length_ns * 1e6 / timestep_fs)

## Step 4 - Visualize/Analyze your MD! Fast RDF generation

#### Loading the trajectory 

In [ ]:
traj = ase.io.read(trajectory_path, index=":")
# Skip first time-step which is just the initial configuration
traj = traj[1:]
print(f"Loaded MD trajectory of length {len(traj)}")

#### Fetching RDF values from trajectory

In [ ]:
equilibration_fs = 0.5 * 100  # deliberately short
nbins_per_angstrom = 50

# Skip the beginning of the trajectory where simulation is not at equilibrium
traj_len_fs = timestep_fs * len(traj)
# equilibration time / 100 (sampling interval)
traj_eq = traj[int(equilibration_fs / timestep_fs / 100) :]

rmax = 6.3
nbins = int(rmax * nbins_per_angstrom)

element_pair = (8, 8)  # O-O distances

# Compute the average RDF
rdf_list = []
for i, atoms in enumerate(traj_eq):
    rdf, radii = get_rdf(
        atoms,
        rmax,
        nbins,
        no_dists=False,
        elements=element_pair,
    )
    rdf_list.append(rdf)
avg_rdf = np.mean(rdf_list, axis=0)

#### Plotting the results!

In [ ]:
fig, ax = plt.subplots()
ax.plot(radii, avg_rdf, lw=3)
ax.set_xlabel("Radius (A)")
ax.set_ylabel("g(r)")
ax.set_title(f"O-O RDF of water at {temperature_K}K")
ax.set_xlim([1.5, rmax])